# NETFLIX 
by Aurélie RAOUL

In [63]:
# Import nécessaires
from time import time
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns

verbose = True

## 1. Lecture du fichier

In [2]:
print("Chargement des données....")
df = pd.read_csv('netflix_titles.csv', sep=',')
print("Chargement des données........ END")

Chargement des données....
Chargement des données........ END


## 2. Dimension du dataframe

In [3]:
print("Dimension du dataframe", df.shape)

Dimension du dataframe (8807, 12)


In [4]:
print("Combien par type:\n", df["type"].value_counts())

Combien par type:
 Movie      6131
TV Show    2676
Name: type, dtype: int64


In [5]:
def get_nb_of_type(df, type, verbose=False):
    """
    Retourne le nombre d'éléments du type demandé en paramètre
    :param df: DataFrame
    :param type : String - type recherché
    :param verbose : Boolean - True pour mode debug
    :return: le nombre d'éléments du type demandé en paramètre
    """
    nb_type = 0
    types_count = df["type"].value_counts()
    for t,v in types_count.items():
        if t == type:
            nb_type = v
            break
    return nb_type

In [6]:
print("Combien y a-t-il de films dans ce jeu de données ?", get_nb_of_type(df, "Movie", verbose))

Combien y a-t-il de films dans ce jeu de données ? 6131


In [7]:
print("Combien y a-t-il de séries ?", get_nb_of_type(df, "TV Show", verbose))

Combien y a-t-il de séries ? 2676


In [8]:
print("Générer le résumé statistique du dataframe")
print(df.describe())

Générer le résumé statistique du dataframe
       release_year
count   8807.000000
mean    2014.180198
std        8.819312
min     1925.000000
25%     2013.000000
50%     2017.000000
75%     2019.000000
max     2021.000000


## 3. Nettoyge des données

#### 3.1 Correction des types

In [9]:
def cleanType(df, verbose=False):
    """
    Nettoie et Transforme les types : type, rating, date_added, duration
    :param df: DataFrame
    :param verbose : Boolean - True pour mode debug
    :return: DataFrame - a new clean DataFrame
    """
    print("cleanType ....")
    df = df.copy()

    # Traitement du type
    df["type"] = df["type"].astype('category')

    # Traitement du rating
    if verbose: print("rating : ", df["rating"].unique())
    # Il y a un décalage pour certaines notes, donc il faut les corriger avant de changer le type
    for rating in df["rating"].unique():
        if " min" in str(rating):
            # Il faut corriger
            df.loc[df["rating"] == rating, 'duration'] = rating
            df.loc[df["rating"] == rating, 'rating'] = np.nan
    if verbose: print("rating : ", df["rating"].unique())
    # Conversion des notes en catégorie
    df["rating"] = df["rating"].astype('category')

    # Traitement de la date
    df["date_added"] = pd.to_datetime(df['date_added'])

    # Traitement de la duration
    df["duration time"] = df["duration"].str.replace(' min','')
    df["duration time"] = df["duration time"].str.replace(' Seasons', '')
    df["duration time"] = df["duration time"].str.replace(' Season', '')
    if verbose:
        print(df[["duration","duration time"]])
        print(df.dtypes)
    df["duration time"] = pd.to_numeric(df["duration time"])
    print("cleanType.................... END")
    if verbose: print(df.dtypes)
    return df

In [10]:
# Correction des types
print(df.dtypes)
df = cleanType(df, verbose)
print(df.dtypes)
print(df.describe())
print("Compter les valeurs manquantes")
print(df.isna().sum())

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object
cleanType ....
rating :  ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' nan 'TV-Y7-FV' 'UR']
rating :  ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' nan 'NR' 'TV-Y7-FV' 'UR']
       duration duration time
0        90 min            90
1     2 Seasons             2
2      1 Season             1
3      1 Season             1
4     2 Seasons             2
...         ...           ...
8802    158 min           158
8803  2 Seasons             2
8804     88 min            88
8805     88 min            88
8806    111 min           111

[8807 rows x 2 columns]
show_id                  object
type                   ca

#### 3.2 Traitement des duplicates

In [11]:
def cleanDuplicated(df, verbose=False):
    """
    Delete duplicate datas
    :param df: DataFrame
    :param verbose : Boolean - True pour mode debug
    :return: DataFrame - a new clean DataFrame
    """
    print("cleanDuplicated ....")
    df = df.copy()
    # Existe-t-il des doublons dans ce jeu de données ?
    if verbose:
        print(df.nunique())
        print(df.duplicated())
        print("df.duplicated().sum() =", df.duplicated().sum())
        print("show_id")
        print(df.shape, df["show_id"].nunique())
        print(df["show_id"].duplicated().sum())
    print("cleanDuplicated ................... END")
    return df

In [12]:
df = cleanDuplicated(df, verbose)

cleanDuplicated ....
show_id          8807
type                2
title            8807
director         4528
cast             7692
country           748
date_added       1714
release_year       74
rating             14
duration          220
listed_in         514
description      8775
duration time     210
dtype: int64
0       False
1       False
2       False
3       False
4       False
        ...  
8802    False
8803    False
8804    False
8805    False
8806    False
Length: 8807, dtype: bool
df.duplicated().sum() = 0
show_id
(8807, 13) 8807
0
cleanDuplicated ................... END


#### 3.3 Traitement des données manquantes

In [13]:
print("Sur la colonne des directeurs de production", df["director"].isna().sum())
print("Sur la colonne des acteurs", df["cast"].isna().sum())
print("Suppression des lignes dupliquées (s'ils existents)")

Sur la colonne des directeurs de production 2634
Sur la colonne des acteurs 825
Suppression des lignes dupliquées (s'ils existents)


### Conclusion nettoyage
Plusieurs types de données à traiter et plusieurs corrections de données
Pas de nettoyage nécessaire sur les doublons ou les données manquantes

## 4. Questions


In [15]:
def process_country(df, country, verbose=False):
    df = df.copy()
    df[country] = df["country"].str.contains(country, case=False)
    if verbose:
        nb_total = df[country].value_counts()[True]
        select = df[df["type"] == "Movie"].index.intersection(df[df[country] == True].index)
        nb_films = len(select)
        select = df[df["type"] == "TV Show"].index.intersection(df[df[country] == True].index)
        nb_series = len(select)
        print(country, "nb_total :", nb_total, " - nb_films :", nb_films, " - nb_series :", nb_series)
    return df

Combien de films/séries ont été produit par les Etats-Unis ? Par la France ?

In [17]:
# Identification des pays via ajout de colonne
states = 'United States'
france = 'France'

df = process_country(df, states, True)
df = process_country(df, france, True)

# Films en colaboration France / Etats Unis
select = df[df[states] == True].index.intersection(df[df[france] == True].index)
print("Films en colaboration France / Etats Unis :", len(select.value_counts()))

United States nb_total : 3690  - nb_films : 2752  - nb_series : 938
France nb_total : 393  - nb_films : 303  - nb_series : 90
Films en colaboration France / Etats Unis : 125


Quel est le film avec la durée la plus longue sur Netflix ?

In [23]:
select = df[df["type"] == "Movie"]
duree = select["duration time"].max()
print("le film avec la durée la plus longue (", duree, ")")
select[select["duration time"] == duree]

le film avec la durée la plus longue ( 312 )


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration time,United States,France
4253,s4254,Movie,Black Mirror: Bandersnatch,NaN,"Fionn Whitehead, Will Poulter, Craig Parkinson...",United States,2018-12-28,2018,TV-MA,312 min,"Dramas, International Movies, Sci-Fi & Fantasy","In 1984, a young programmer begins to question...",312,True,False


## 5. Etude des catégories avec le plus de contenu

In [38]:
# Construction de la liste des catégories
categories_list = {}
#if verbose: print("listed_in : ", df["listed_in"].unique())
for cats_str in df["listed_in"].unique():
    cats = cats_str.split(", ")
    for cat in cats:
        if cat not in categories_list:
            # comptage du nombre d'élément par catégorie
            nb = count_categorie(df, cat, verbose)
            categories_list[cat] = nb
print("Nb Catégories :", len(categories_list))
#print("Catégories :", categories_list)
categories_list

Nb Catégories : 42


{'Documentaries': 869,
 'International TV Shows': 1351,
 'TV Dramas': 763,
 'TV Mysteries': 98,
 'Crime TV Shows': 470,
 'TV Action & Adventure': 168,
 'Docuseries': 395,
 'Reality TV': 255,
 'Romantic TV Shows': 370,
 'TV Comedies': 581,
 'TV Horror': 75,
 'Children & Family Movies': 641,
 'Dramas': 3190,
 'Independent Movies': 756,
 'International Movies': 2752,
 'British TV Shows': 253,
 'Comedies': 2255,
 'Spanish-Language TV Shows': 174,
 'Thrillers': 634,
 'Romantic Movies': 616,
 'Music & Musicals': 375,
 'Horror Movies': 357,
 'Sci-Fi & Fantasy': 327,
 'TV Thrillers': 57,
 "Kids' TV": 451,
 'Action & Adventure': 1027,
 'TV Sci-Fi & Fantasy': 84,
 'Classic Movies': 116,
 'Anime Features': 71,
 'Sports Movies': 219,
 'Anime Series': 176,
 'Korean TV Shows': 151,
 'Science & Nature TV': 92,
 'Teen TV Shows': 69,
 'Cult Movies': 71,
 'TV Shows': 1755,
 'Faith & Spirituality': 65,
 'LGBTQ Movies': 102,
 'Stand-Up Comedy': 399,
 'Movies': 4502,
 'Stand-Up Comedy & Talk Shows': 56,
 '

In [48]:
def count_unique_data(df, column_name, cat, verbose=False):
    nb_total = 0
    serie = df[column_name].str.contains(cat, case=False)
    nb_total = serie.value_counts()[True]
    return nb_total


def get_unique_data(df, column_name, verbose=False):
    categories_list = {}
    categories_by_size = {}
    max = 0
    if verbose: print(column_name, " : ", df[column_name].unique())
    for cats_str in df[column_name].unique():
        if cats_str is None or cats_str == np.nan or type(cats_str) != str:
            continue
        cats = cats_str.split(", ")
        for cat in cats:
            if cat not in categories_list:
                # comptage du nombre d'élément par catégorie
                nb = count_unique_data(df, column_name, cat, verbose)
                categories_list[cat] = nb
                if not categories_by_size.get(nb, False):
                    categories_by_size[nb] = []
                categories_by_size[nb].append(cat)
                if verbose:
                    print(cat," : " , nb)
                if nb > max:
                    max = nb
    return categories_list, categories_by_size, max

### 5.1 Etude des catégories avec le plus de contenu.

In [64]:
verbose = False
# Construction de la liste des catégories
t0 = time()
categories_list, categories_by_size, max2 = get_unique_data(df, "listed_in", verbose)
t1 = time() - t0
#print("Catégories :", categories_list)
print("Catégories avec le plus de contenus :", categories_by_size[max2], "avec :", max2, "contenus")
# # La catégorie "Movies" n'est pas intéressante
# categories_by_size.pop(max2, None)
# max_after_movie = max(categories_by_size.keys())
# print(max_after_movie)
# print(categories_by_size)
# print(categories_by_size[max_after_movie])
keys = sorted(categories_by_size.keys(), reverse=True)
for k in keys:
    print(k, " pour ", categories_by_size[k])

Catégories avec le plus de contenus : ['Movies'] avec : 4502 contenus
4502  pour  ['Movies']
3190  pour  ['Dramas']
2752  pour  ['International Movies']
2255  pour  ['Comedies']
1755  pour  ['TV Shows']
1351  pour  ['International TV Shows']
1027  pour  ['Action & Adventure']
869  pour  ['Documentaries']
763  pour  ['TV Dramas']
756  pour  ['Independent Movies']
641  pour  ['Children & Family Movies']
634  pour  ['Thrillers']
616  pour  ['Romantic Movies']
581  pour  ['TV Comedies']
470  pour  ['Crime TV Shows']
451  pour  ["Kids' TV"]
399  pour  ['Stand-Up Comedy']
395  pour  ['Docuseries']
375  pour  ['Music & Musicals']
370  pour  ['Romantic TV Shows']
357  pour  ['Horror Movies']
327  pour  ['Sci-Fi & Fantasy']
255  pour  ['Reality TV']
253  pour  ['British TV Shows']
219  pour  ['Sports Movies']
176  pour  ['Anime Series']
174  pour  ['Spanish-Language TV Shows']
168  pour  ['TV Action & Adventure']
151  pour  ['Korean TV Shows']
116  pour  ['Classic Movies']
102  pour  ['LGBTQ Mo

La catégorie "Movies" n'est pas intéressante, possibilité de supprimer cette donnée par la suite si besoin

### 5.2 Quels directeurs ont produit le plus de films/séries disponibles sur Netflix ?

In [61]:
verbose = False
t0 = time()
# Construction de la liste des catégories
directors_list, directors_by_size, max_dir = get_unique_data(df, "director", verbose)
t1 = time() - t0
print("Nb directors :", len(directors_list), '{0:.3f} secondes'.format(t1))
print("Directors avec le plus de contenus :", directors_list[max_dir], "avec :", max_dir, "contenus")
keys = sorted(directors_by_size.keys(), reverse=True)

# Pour éviter de tout afficher
i = 5
if verbose:
    i = len(keys)
for k in keys:
    print(k, " pour ", directors_by_size[k])
    i -= 1
    if i == 0:
        break

Nb directors : 4993
Directors avec le plus de contenus : ['Ram'] avec : 78 contenus
78  pour  ['Ram']
22  pour  ['Rajiv Chilaka']
21  pour  ['Jan Suter']
20  pour  ['Raúl Campos']
16  pour  ['Suhas Kadav', 'Marcus Raboy']


### 5.3. Est-ce que Jan Suter travaille souvent avec les mêmes acteurs ?

In [67]:
verbose = False
print("Est-ce que Jan Suter travaille souvent avec les mêmes acteurs ?")
director_name = "Jan Suter"
df = df.copy()
if verbose:
    print("james_suter - BEFORE drop", df.shape)
df = df.dropna(subset=["director"])
if verbose:
    print("james_suter - AFTER drop", df.shape)
select = df[df["director"].str.contains(director_name, case=False)]
t0 = time()
# Construction de la liste des catégories
actors_list, actors_by_size, max_dir = get_unique_data(select, "cast", verbose)
t1 = time() - t0
print("Nb actors :", len(actors_list), 'in {0:.3f} secondes'.format(t1))
print("Directors avec le plus de contenus :", actors_by_size[max_dir], "avec :", max_dir, "contenus")
keys = sorted(actors_by_size.keys(), reverse=True)

for k in keys:
    print(k, " pour ", actors_by_size[k])

Est-ce que Jan Suter travaille souvent avec les mêmes acteurs ?
Nb actors : 22 0.044 secondes
Directors avec le plus de contenus : ['Carlos Ballarta', 'Sofía Niño de Rivera'] avec : 3 contenus
3  pour  ['Carlos Ballarta', 'Sofía Niño de Rivera']
2  pour  ["Ricardo O'Farrill"]
1  pour  ['Coco Celis', 'Raúl Meneses', 'Luciano Mellera', 'Jani Dueñas', 'Fernando Sanjiao', 'Lucas Lauriente', 'Malena Pichot', 'Natalia Valdebenito', 'Sebastián Marcelo Wainraich', 'Ricardo Quevedo', 'Julián Arango', 'Antonio Sanint', 'Alexis de Anda', 'Alex Fernández', 'Alan Saldaña', 'Manu NNa', 'Daniel Sosa', 'Chumel Torres', 'Marissa "Chabe" Lozano']


Est-ce que Jan Suter travaille souvent avec les mêmes acteurs ?

Combien de films/séries ont été produit par les Etats-Unis ? Par la France ?

Combien de films/séries ont été produit par les Etats-Unis ? Par la France ?